In [1]:
from struct import unpack
import numpy as np
import matplotlib.pylab as plt
import gzip
from scipy.stats import multivariate_normal
from sklearn.svm import SVC, LinearSVC

def loadmnist(imagefile, labelfile):

    # Open the images with gzip in read binary mode
    images = gzip.open(imagefile, 'rb')
    labels = gzip.open(labelfile, 'rb')

    # Get metadata for images
    images.read(4)  # skip the magic_number
    number_of_images = images.read(4)
    number_of_images = unpack('>I', number_of_images)[0]
    rows = images.read(4)
    rows = unpack('>I', rows)[0]
    cols = images.read(4)
    cols = unpack('>I', cols)[0]

    # Get metadata for labels
    labels.read(4)
    N = labels.read(4)
    N = unpack('>I', N)[0]

    # Get data
    x = np.zeros((N, rows*cols), dtype=np.uint8)  # Initialize numpy array
    y = np.zeros(N, dtype=np.uint8)  # Initialize numpy array
    for i in range(N):
        for j in range(rows*cols):
            tmp_pixel = images.read(1)  # Just a single byte
            tmp_pixel = unpack('>B', tmp_pixel)[0]
            x[i][j] = tmp_pixel
        tmp_label = labels.read(1)
        y[i] = unpack('>B', tmp_label)[0]

    images.close()
    labels.close()
    return (x, y)

def displaychar(image):
    plt.imshow(np.reshape(image, (28,28)), cmap=plt.cm.gray)
    plt.axis('off')
    plt.show()

In [2]:
trainImage, trainLabel = loadmnist('train-images-idx3-ubyte.gz', 'train-labels-idx1-ubyte.gz')
testImage, testLabel = loadmnist('t10k-images-idx3-ubyte.gz', 't10k-labels-idx1-ubyte.gz')

In [ ]:
#print trainImage

In [ ]:
# LinearSVM Q8.A
for i in range(-2,3): 
    clf = LinearSVC(loss='hinge', C= float(10**i))
    clf.fit(trainImage,trainLabel)
    
    train_error = 0
    train_prediction = clf.predict(trainImage)
    
    test_error = 0
    test_prediction = clf.predict(testImage)
    
    for b in range(len(train_prediction)):
        if(train_prediction[b] != trainLabel[b]):
            train_error += 1
    for a in range(len(test_prediction)):
        if(test_prediction[a] != testLabel[a]):
            test_error += 1
    print "The C value is " + str(10**i) + " and the Train error rate is " + str(float(train_error)/len(trainLabel))
    print "And the test error is " + str(float(test_error)/len(testLabel))
    
    
'''
delta = 0.005
x1_min, x1_max = 1.5, 4.5
x2_min, x2_max = 0.0, 2.75
xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, delta), np.arange(x2_min, x2_max, delta))
Z = clf.predict(np.c_[xx1.ravel(), xx2.ravel()])
Z = Z.reshape(xx1.shape)
plt.pcolormesh(xx1, xx2, Z, cmap=plt.cm.Pastel2, vmin=0, vmax=2)
'''

In [5]:
# KernelSVM Q8.B
clf1 = SVC(kernel='poly', degree=2, C= 1)
clf1.fit(trainImage,trainLabel)

train_error1 = 0
train_prediction1 = clf1.predict(trainImage)

test_error1 = 0
test_prediction1 = clf1.predict(testImage)

for b in range(len(train_prediction1)):
    if(train_prediction1[b] != trainLabel[b]):
        train_error1 += 1
for a in range(len(test_prediction1)):
    if(test_prediction1[a] != testLabel[a]):
        test_error1 += 1


In [10]:
print train_error1
print "The C value is " + str(1) + " and the Train error rate is " + str(float(train_error1)/len(trainLabel))
print "And the test error is " + str(float(test_error1)/len(testLabel))
print clf1.n_support_[0] + clf1.n_support_[1]
print clf1.n_support_

0
The C value is 1 and the Train error rate is 0.0
And the test error is 0.0194
899
[ 476  423  984  989  876 1060  641  882 1186 1135]


In [11]:
sum1 = 0
for a in clf1.n_support_:
    sum1 += a
print sum1

8652


In [ ]:

import numpy as np
from sklearn.svm import SVC, LinearSVC
import matplotlib.pyplot as plt

# Now load IRIS data
from sklearn import datasets
iris = datasets.load_iris()
x = iris.data
y = iris.target
 
# Now select two of the four features
f1,f2 = 1,3
features = [f1,f2]
 
# Now train SVM
clf = LinearSVC(loss='hinge', multi_class='crammer_singer')
clf.fit(x[:,features],y)
 
# Create mesh for plot
delta = 0.005
x1_min, x1_max = 1.5, 4.5
x2_min, x2_max = 0.0, 2.75
xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, delta), np.arange(x2_min, x2_max, delta))
Z = clf.predict(np.c_[xx1.ravel(), xx2.ravel()])
Z = Z.reshape(xx1.shape)
plt.pcolormesh(xx1, xx2, Z, cmap=plt.cm.Pastel2, vmin=0, vmax=2)
 
# Plot also the training points
cols = ['ro', 'k^', 'b*']
for label in range(0,3):
    plt.plot(x[y==label,f1], x[y==label,f2], cols[label], markersize=8)
plt.xlabel(iris.feature_names[f1], fontsize=14)
plt.ylabel(iris.feature_names[f2], fontsize=14)
plt.xlim(x1_min, x1_max)
plt.ylim(x2_min, x2_max)
plt.show()
